<div align="justify"><h1><b><font size="6">
  Input Assumptions for Modelling the Building Sector in the negaWatt-BE Scenario</h1></font></div>
<div align="justify"><h2><b><font size="5">
  ENERGY DEMAND FOR THE BUILDING SECTOR IN BELGIUM
</h2></font></b></div>

---
<div align="justify"><h3><font size="3">
  <b> Developer:</b> LATERRE Antoine, BERNAERTS Valentine, QUOILIN Sylvain, MEYER Sébastien
</h3></font></div>
    
---


To ensure transparency and consistency within the negaWatt-BE project, this notebook documents all key input assumptions related to the building sector. It computes the energy demand of the residential and tertiary sectors under sufficiency and efficiency measures over the energy transition period (2020–2050). Projections are based on statistical data from the starting reference year, and the resulting energy demand values are directly used as inputs for the PyPSA model.

> **Note:** The reference year is 2019 instead of 2020, due to the disruptions caused by the COVID-19 pandemic in 2020. This choice ensures greater consistency and realism in the baseline and future projections.


<font size="4"><b>Table of Contents</b></font><a class="anchor" id="contents"></a> <br>

<font size="4"><b>References</b></font><a class="anchor" id="references"></a> <br>

* [1] Rozsai, Mate; Jaxa-Rozen, Marc; Salvucci, Raffaele; Sikora, Przemyslaw; Gea Bermudez, Juan; Neuwahl, Frederik (2025). JRC-IDEES-2023. European Commission, Joint Research Centre (JRC). [Dataset].<br>
  PID: http://data.europa.eu/89h/1f0b480c-6d21-4d95-897d-20c7ca33df6f
* [2] Rozsai, Mate; Jaxa-Rozen, Marc; Salvucci, Raffaele; Sikora, Przemyslaw; Tattini, Jacopo; Neuwahl, Frederik (2024). JRC-IDEES-2021: the Integrated Database of the European Energy System – Data update and technical documentation, Publications Office of the European Union, Luxembourg, 2024. [Report].<br>

<font size="4"><b>Contacts and experts in the field</b></font><a class="anchor" id="contacts"></a>

* Marc Jaxa-Rozen: person in charge of the JRC-IDEES database at the Joint Research Centre (JRC-C6-JRC-IDEES@ec.europa.eu).
* Mate Rozsai: person in charge of the JRC-IDEES database at the Joint Research Centre (JRC-C6-JRC-IDEES@ec.europa.eu).

<font size="4"><b>Loading external functions</b></font><a class="anchor" id="project_functions"></a>

The functions required for this notebook are loaded here. The macroscopic data (demographic projections, etc.) contained in `nW_BE_demand_model_macro.ipynb` are also loaded here.

In [1]:
# Automatically reload the file if it is modified:
%load_ext autoreload
%autoreload 2
# Load the macro parameters and necessary packages:
%run ./nW_BE_demand_model_macro.ipynb
# Option to print and plot the results:
post_process = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

<font size="5"><b>1. Introduction</b></font> <a class="anchor" id="section_1"></a>

---

The method used to generate the energy demand projections for the building sector follows the approach of the Joint Research Centre (JRC) of the European Commission for building the Integrated Database of the European Energy System (IDEES) [1,2]. This database compiles statistics related to the residential and tertiary sectors.

---

<font size="4"><b>1.1. Sufficiency vs Efficiency</b></font> <a class="anchor" id="section_1_1"></a>

---

To reduce primary energy consumption in the building sector, the negaWatt-BE approach relies on two complementary strategies: energy sufficiency, acting on usefull energy demand, and energy efficiency, acting on conversion losses. These strategies are implemented through several levers, presented below, which guide the transition towards a low-energy building sector.
* $\textcolor{Orange}{\textbf{Energy Sufficiency}}$ refers to a reduction in the final energy demand and energy services. This is achieved through behavioral and systemic changes, including a reduction in the heating temperature setting.
* $\textcolor{Orange}{\textbf{Energy Efficiency}}$ refers to technical improvements that reduce the amount of primary energy consumed per building (like building insulation), without requiring behavioral or 'comfort' changes.


---

<font size="4"><b>1.2. Global Demands</b></font> <a class="anchor" id="section_1_2"></a>

---

---

<font size="3"><b>1.2.1. Residential Sector</b></font> <a class="anchor" id="section_1_2_1"></a>

---

According to JRC-IDEES [1,2], the total residential surface area was 625,236 Mm² in 2019. 
Using the population size and average household size from Stabel (see `nW_BE_demand_model_macro.ipynb`), we get the specific residential surface area and the average residential household surface area.)..

In [2]:
# Inputs - Define Sufficiency Scenario Data (SUF)
ref_RS_sur_tot = 625235.722685535*1e+3
ref_RS_sur_spe = ref_RS_sur_tot/population_dict[2019] # [m²/person]
ref_RS_sur_hld = ref_RS_sur_tot/households_dict[2019] # [m²/household]


> **Comment:** Households do actually not represent proper dwellings. We follow the same approach as JRC-IDEES [2]: we assign them implicitly to households without distinguishing secondary residences or vacant dwellings.

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

As a global sufficiency assumption, we want to reduce the specific residential surface area. 
Instead, we do not play on the average household size, which is a demographic projection from Statbel (see `nW_BE_demand_model_macro.ipynb`).
In light of this, we project a **-10% reduction** in residential surface area available per person.


In [3]:
# Inputs - Define Sufficiency Scenario Data (SUF)
pro_RS_sur_spe = -0.10
# Outputs - Sufficiency Scenario Data (SUF)
SUF_data = {"RS specific surface [m²/person]": linear_growth(2019, ref_RS_sur_spe, 
                                                             2050, ref_RS_sur_spe*(1+pro_RS_sur_spe), years)}
df_SUF = pd.DataFrame(SUF_data, index=years)
df_SUF["population [person]"]                 = df_SUF.index.map(population_dict)
df_SUF["households [household]"]              = df_SUF.index.map(households_dict)
df_SUF["RS total surface [Mm²]"]              = df_SUF["RS specific surface [m²/person]"]*df_SUF["population [person]"]*1e-6
df_SUF["RS household surface [m²/household]"] = df_SUF["RS total surface [Mm²]"]*1e+6/df_SUF.index.map(households_dict)

---

<font size="3"><b>1.2.2. Tertiary Sector</b></font> <a class="anchor" id="section_1_2_2"></a>

---


In [4]:
# Inputs - Define Sufficiency Scenario Data (SUF)
ref_TS_sur_tot = 227073.149760375*1e+3
ref_TS_sur_spe = ref_TS_sur_tot/population_dict[2019] # [m²/person]


<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

blabla

In [5]:
# Inputs - Define Sufficiency Scenario Data (SUF)
pro_TS_sur_spe = -0.10
# Outputs - Sufficiency Scenario Data (SUF)
df_SUF["TS specific surface [m²/person]"]     = linear_growth(2019, ref_TS_sur_spe, 
                                                              2050, ref_TS_sur_spe*(1+pro_TS_sur_spe), years)
df_SUF["TS total surface [Mm²]"]              = df_SUF["TS specific surface [m²/person]"]*df_SUF["population [person]"]*1e-6

---

<font size="3"><b>1.2.3. Data Processing</b></font> <a class="anchor" id="section_1_2_3"></a>

---


In [6]:
# Processing - Sectors
df_BD_macro = {
    'Demographic':   df_SUF[["population [person]","households [household]"]].T,
    'Residential':   df_SUF[["RS total surface [Mm²]","RS household surface [m²/household]","RS specific surface [m²/person]"]].T,
    'Tertiary':      df_SUF[["TS total surface [Mm²]","TS specific surface [m²/person]"]].T,
}
rows = []
for sector, df in df_BD_macro.items():
    temp = df.copy()
    temp['Sector'] = sector
    temp['Parameter'] = temp.index
    temp = temp.reset_index(drop=True)
    rows.append(temp)
df_BD_macro = pd.concat(rows, ignore_index=True)

In [7]:
if post_process:
    # === Full table ===
    df_fec = df_BD_macro
    df_fec['Sector'] = df_fec['Sector'].mask(df_fec['Sector'].duplicated(), '')
    df_fec_r = df_fec[['Sector', 'Parameter'] + list(years)]
    styled = (
        df_fec_r.style
          .hide(axis='index')
          .set_table_attributes('style="width:100%;table-layout:fixed;"')
          .apply(highlight_mode_separator, axis=1)
          .apply(lambda row: [bold_mode(cell, row['Sector'], col) for col, cell in zip(df_fec_r.columns, row)], axis=1)
          .set_properties(subset=['Parameter'], **{'font-style':'italic'})
          .format({year: "{:.1f}" for year in years})
    )
    display(styled)

Sector,Parameter,2019,2025,2030,2035,2040,2045,2050
Demographic,population [person],11431406.0,11816102.0,12023862.0,12186730.0,12347171.0,12490658.0,12600911.0
,households [household],4948398.0,5199667.0,5355123.0,5489927.0,5609698.0,5702818.0,5770867.0
Residential,RS total surface [Mm²],625.2,633.8,634.3,632.2,629.6,625.9,620.3
,RS household surface [m²/household],126.4,121.9,118.4,115.1,112.2,109.7,107.5
,RS specific surface [m²/person],54.7,53.6,52.8,51.9,51.0,50.1,49.2
Tertiary,TS total surface [Mm²],227.1,230.2,230.4,229.6,228.6,227.3,225.3
,TS specific surface [m²/person],19.9,19.5,19.2,18.8,18.5,18.2,17.9



---

<font size="5"><b>2. Residential Sector</b></font> <a class="anchor" id="section_2"></a>

---



---

<font size="4"><b>2.1. Thermal Uses</b></font> <a class="anchor" id="section_2_1"></a>

---


Distribution of different energy services for thermal uses the reference year


In [8]:
# Inputs - Repartition of the thermal energy services (TES) in residential buildings for the reference year (2019)
ref_RS_tes_sht = 3664.049*ktoe_to_GWh*1e+6/ref_RS_sur_tot # [kWh/m²]
ref_RS_tes_scl =   23.455*ktoe_to_GWh*1e+6/ref_RS_sur_tot # [kWh/m²]
ref_RS_tes_dhw =  663.435*ktoe_to_GWh*1e+6/df_SUF["population [person]"][2019] # [kWh/person]
ref_RS_tes_cok =   96.395*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
print(f'Energy service for space heating: {round(ref_RS_tes_sht,3)} kWh/m²')
print(f'Energy service for space cooling: {round(ref_RS_tes_scl,3)} kWh/m²')
print(f'Energy service for hot water:     {round(ref_RS_tes_dhw,3)} kWh/person')
print(f'Energy service for cooking:       {round(ref_RS_tes_cok,3)} kWh/household')

Energy service for space heating: 68.155 kWh/m²
Energy service for space cooling: 0.436 kWh/m²
Energy service for hot water:     674.961 kWh/person
Energy service for cooking:       226.553 kWh/household



---

<font size="3"><b>2.1.1. Building Energy Performance</b></font> <a class="anchor" id="section_2_1_1"></a>

---

Split old and new/ren?

For old: keep it more or less constant

For new: improve?

De 2001 à 2023, le taux de modernisation du parc de bâtiments (rénovation + nouvelle construction) est de l'ordre de 3%/an (m²_ren/m²_tot).

<br/>
$\textcolor{Orange}{\textbf{Efficiency Assumption}}$

Renovation rate from 3% to 4%

In [9]:
acc_RS_tes_sht_ren = 2
cur_RS_tes_sht_ren = -0.458 # kWh/m²/year
trg_RS_tes_sht     = ref_RS_tes_sht + acc_RS_tes_sht_ren*cur_RS_tes_sht_ren*(2050-2019) # [kWh/m²]


---

<font size="3"><b>2.1.2. Behavioural Aspects</b></font> <a class="anchor" id="section_2_1_2"></a>

---

**Space heating** 

adoption of SlowHeating, report some to electric appliances!

In [10]:
d_cons_temp = 0.07
d_temp = 2
suf_RS_tes_sht = 1-d_temp*d_cons_temp

**Space cooling** 

limit development by maintening historical pace 

In [11]:
acc_RS_tes_scl_ren = 1
cur_RS_tes_scl_ren = 0.035 # kWh/m²/year
trg_RS_tes_scl     = ref_RS_tes_scl + acc_RS_tes_scl_ren*cur_RS_tes_scl_ren*(2050-2019) # [kWh/m²]

**Domestic hot water**

By 2019, the specific annual energy consumption for hot water is of about 675 kWh/person. This is equivalent to about 64 l of 40°C per day and per person.

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

We project to limit the hot water consumption for showering to sufficient levels. We therefore assume one shower per day and per person, 5 min long shower (sinking water) with 38°C water at 7 l/min. We also consider a daily use of 10 l of 60°C hot water for other domestic purposes (doing the dish, etc.). 

All in all, the specific energy consumption by 2050 is reduced by **-21%** compared with 2019. This is equivalent to 50 l of 40°C per day and per person.

In [12]:
# Inputs - Define Sufficiency Scenario Data (SUF)
shower_duration    = 5 # [min]
shower_flow_rate   = 7 # [l/min]
shower_temperature = 38 # [°C]
shower_energy      = shower_duration*shower_flow_rate*rho_h2o*cp_h2o*(shower_temperature-15) # [kWh/shower]
others_volume      = 10 # [l]
others_temperature = 60 # [°C]
others_energy      = others_volume*rho_h2o*cp_h2o*(others_temperature-15) # [kWh]
# Outputs - Sufficiency Scenario Data (SUF)
trg_RS_tes_dhw     = (shower_energy+others_energy)*365
pro_RS_tes_dhw     = (trg_RS_tes_dhw-ref_RS_tes_dhw)/ref_RS_tes_dhw
print(f'Evolution of domestic hot water consumption: {round(pro_RS_tes_dhw*100,1)}% [kWh/person]')
amount_2019 = ref_RS_tes_dhw/(rho_h2o*cp_h2o*(40-15))/365 # equivalent l/day/person of 40°C water
amount_2050 = trg_RS_tes_dhw/(rho_h2o*cp_h2o*(40-15))/365 # equivalent l/day/person of 40°C water
print(f' -> By 2019, the equivalent consumption is {round(amount_2019,1)} liters of 40°C water per day and per person')
print(f' -> By 2050, the equivalent consumption is {round(amount_2050,1)} liters of 40°C water per day and per person')

Evolution of domestic hot water consumption: -21.2% [kWh/person]
 -> By 2019, the equivalent consumption is 63.7 liters of 40°C water per day and per person
 -> By 2050, the equivalent consumption is 50.2 liters of 40°C water per day and per person


**Cooking**

15% increase, people cook more.



drop gas, go full electric


In [13]:
# Inputs - Define Sufficiency Scenario Data (SUF)
pro_RS_tes_cok     = 0.15
ref_RS_tes_cok_gas = (3.846+21.957)/96.395 # ktoe_gas/ktoe_tot (from JRC-IDEES)
trg_RS_tes_cok_gas = 0.02
# Outputs - Sufficiency Scenario Data (SUF)
trg_RS_tes_cok     = (1+pro_RS_tes_cok)*ref_RS_tes_cok # kWh/person
#trg_RS_tes_cok_gas = (1+pro_RS_tes_cok)*ref_RS_tes_cok_gas 
print(f'Evolution of cooking consumption: {round(pro_RS_tes_cok*100,1)}% [kWh/household]')
print(f' -> By 2019, the equivalent consumption is {round(ref_RS_tes_cok/365,2)} kWh per day and per household')
print(f' -> By 2050, the equivalent consumption is {round(trg_RS_tes_cok/365,2)} kWh per day and per household')

Evolution of cooking consumption: 15.0% [kWh/household]
 -> By 2019, the equivalent consumption is 0.62 kWh per day and per household
 -> By 2050, the equivalent consumption is 0.71 kWh per day and per household



---

<font size="3"><b>2.1.3. Centralisation Degree</b></font> <a class="anchor" id="section_2_1_3"></a>

---



---

<font size="3"><b>2.1.4. Data Processing</b></font> <a class="anchor" id="section_2_1_4"></a>

---

give the fuel share for the reference year!!!

similar to PATHS 2050 - Scenarios towards a 
carbon-neutral Belgium by 205 (https://perspective2050.energyville.be/sites/paths2050/files/inline-files/Full-Fledged%20Report_1.pdf)

In [14]:
tes_RS_tot = {
    'space heating':      linear_growth(2019,ref_RS_tes_sht                       *df_SUF["RS total surface [Mm²]"][2019]*1e-3,
                                        2050,trg_RS_tes_sht   *suf_RS_tes_sht     *df_SUF["RS total surface [Mm²]"][2050]*1e-3,years), # [TWh] kWh/m² * m²
    'space cooling':      linear_growth(2019,ref_RS_tes_scl                       *df_SUF["RS total surface [Mm²]"][2019]*1e-3,
                                        2050,trg_RS_tes_scl                       *df_SUF["RS total surface [Mm²]"][2050]*1e-3,years),
    'domestic hot water': linear_growth(2019,ref_RS_tes_dhw                       *df_SUF["population [person]"][2019]*1e-9,
                                        2050,trg_RS_tes_dhw                       *df_SUF["population [person]"][2050]*1e-9,years), # [TWh] kWh/person * person 
    'cooking':            linear_growth(2019,ref_RS_tes_cok                       *df_SUF["households [household]"][2019]*1e-9,
                                        2050,trg_RS_tes_cok                       *df_SUF["households [household]"][2050]*1e-9,years), # [TWh] kWh/household * household
    'cooking_ng':         linear_growth(2019,ref_RS_tes_cok   *ref_RS_tes_cok_gas *df_SUF["households [household]"][2019]*1e-9,
                                        2050,trg_RS_tes_cok   *trg_RS_tes_cok_gas *df_SUF["households [household]"][2050]*1e-9,years), # [TWh] kWh/household * household
    'cooking_el':         linear_growth(2019,ref_RS_tes_cok*(1-ref_RS_tes_cok_gas)*df_SUF["households [household]"][2019]*1e-9,
                                        2050,trg_RS_tes_cok*(1-trg_RS_tes_cok_gas)*df_SUF["households [household]"][2050]*1e-9,years), # [TWh] kWh/household * household
}

total_TWh  = tes_RS_tot['space heating'][0]+tes_RS_tot['space cooling'][0]+tes_RS_tot['domestic hot water'][0]+tes_RS_tot['cooking'][0]
total_ktoe = total_TWh*1e+3/ktoe_to_GWh
print(total_ktoe)

# From Dashboard_BE_v5.27_AG.xlsx
# => TOTAL Final USEFUL energy consumption for space heating in the residential sector (secondary and vacant dwellings not or partially included)	
#    * 2020: 49,275	
#    * 2050: 30,786
# => TOTAL Final useful energy consumption of dwellings for cooling
#    * 2020:  1,277
#    * 2050:  1,970
# => Total final energy consumption for domestic hot water
#    * 2020:  9,319
#    * 2050:  4,611
# => Total final energy consumption for domestic cooking
#    * 2020:  1,600
#    * 2050:  1,686
tes_RS_tot

4447.37747205503


{'space heating': [42.613, 38.47, 35.018, 31.566, 28.114, 24.661, 21.209],
 'space cooling': [0.273, 0.403, 0.511, 0.619, 0.727, 0.835, 0.944],
 'domestic hot water': [7.716, 7.52, 7.356, 7.193, 7.029, 6.866, 6.702],
 'cooking': [1.121, 1.195, 1.257, 1.318, 1.38, 1.442, 1.504],
 'cooking_ng': [0.3, 0.248, 0.204, 0.161, 0.117, 0.074, 0.03],
 'cooking_el': [0.821, 0.947, 1.053, 1.158, 1.263, 1.368, 1.473]}

In [15]:
ref_eud_tot = ref_RS_tes_sht*df_SUF["RS total surface [Mm²]"][2019]*1e+6 +\
              ref_RS_tes_scl*df_SUF["RS total surface [Mm²]"][2019]*1e+6 + \
              ref_RS_tes_dhw*population_dict[2019] + \
              ref_RS_tes_cok*households_dict[2019] 

print(ref_eud_tot*1e-6/ktoe_to_GWh)

4447.363656905236



---

<font size="4"><b>2.2. Specific Electrical Uses</b></font> <a class="anchor" id="section_2_2"></a>

---

Distribution of different energy services for electrical uses the reference year


In [16]:
# Inputs - Repartition of the electrical energy services (EES) in residential buildings for the reference year (2019)
ref_RS_ees_frg = 209.814*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
ref_RS_ees_wsh =  64.045*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
ref_RS_ees_dry =  73.446*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
ref_RS_ees_dsh =  59.199*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
ref_RS_ees_tvm = 257.507*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
ref_RS_ees_ict =  96.860*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
ref_RS_ees_lgt = 139.889*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
ref_RS_ees_oth =  97.004*ktoe_to_GWh*1e+6/df_SUF["households [household]"][2019] # [kWh/household]
print(f'Energy service for refrigeration:    {round(ref_RS_ees_frg,3)} kWh/household')
print(f'Energy service for washing machines: {round(ref_RS_ees_wsh,3)} kWh/household')
print(f'Energy service for clothes dryer:    {round(ref_RS_ees_dry,3)} kWh/household')
print(f'Energy service for dish washer:      {round(ref_RS_ees_dsh,3)} kWh/household')
print(f'Energy service for TV and multimedia:{round(ref_RS_ees_tvm,3)} kWh/household')
print(f'Energy service for ICT:              {round(ref_RS_ees_ict,3)} kWh/household')
print(f'Energy service for lights:           {round(ref_RS_ees_lgt,3)} kWh/household')
print(f'Energy service for others:           {round(ref_RS_ees_oth,3)} kWh/household')

Energy service for refrigeration:    493.117 kWh/household
Energy service for washing machines: 150.522 kWh/household
Energy service for clothes dryer:    172.617 kWh/household
Energy service for dish washer:      139.133 kWh/household
Energy service for TV and multimedia:605.207 kWh/household
Energy service for ICT:              227.646 kWh/household
Energy service for lights:           328.775 kWh/household
Energy service for others:           227.984 kWh/household



---

<font size="3"><b>2.2.1. Projections</b></font> <a class="anchor" id="section_2_2_1"></a>

---


**Refrigeration**

In [17]:
trg_RS_ees_frg = 120 # [kWh/household]
yea_RS_ees_frg = 2040

2019+(ref_RS_ees_frg-trg_RS_ees_frg)/(16.967)

2040.9907186757787


> **Comment:** Almost linear degrowth from 2000 to 2023 with about -17 kWh/household/year. Use this to assess year target is reached.

> **Comment:** For target, see this EU labels (https://energy-efficient-products.ec.europa.eu/product-list/fridges-and-freezers_en)

**Washing machines**

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

30°C eco-cycle as a priority!

In [18]:
trg_RS_ees_wsh = 90 # [kWh/household]
yea_RS_ees_wsh = 2030

2019+(ref_RS_ees_wsh-trg_RS_ees_wsh)/(6.277)

2028.6418860078816


> **Comment:** Almost linear degrowth from 2000 to 2023 with about -6.3 kWh/household/year. Use this to assess year target is reached.

> **Comment:** For target, see this EU labels (https://energy-efficient-products.ec.europa.eu/product-list/washing-machines_en)



<br/>
$\textcolor{Orange}{\textbf{Efficiency Assumption}}$

shift from condensing electric and vented electric to condensing heat pumps

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

Limit their use and priviliege natural passive drying!

**Clothes dryers**

In [19]:
trg_RS_ees_dry = 80 # [kWh/household]
yea_RS_ees_dry = 2040

2019+(ref_RS_ees_dry-trg_RS_ees_dry)/(4.17)

2041.2102808530192


> **Comment:** Almost linear degrowth from 2000 to 2023 with about -4.2 kWh/household/year. Use this to assess year target is reached.

> **Comment:** For target, see this EU labels (https://energy-efficient-products.ec.europa.eu/product-list/tumble-dryers_en)


**Dishwashers**


<br/>
$\textcolor{Orange}{\textbf{Efficiency Assumption}}$

blabla

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

still continue to deploy. consumption for new dishwasher in 2020 was 170 kWh/y/equipment. By 2019, we were only at 139 kWh/y/household, meaning that not everyone is equiped.

In [20]:
trg_RS_ees_dsh =  150 # [kWh/household]
yea_RS_ees_dsh = 2050

2019+(ref_RS_ees_dsh-trg_RS_ees_dsh)/(-0.351)

2049.960740905656


> **Comment:** Growth from 2000 to 2023 with about +0.35 kWh/household/year, likely to increased adoption.

> **Comment:** For target, see this EU labels (https://energy-efficient-products.ec.europa.eu/product-list/dishwashers_en)


**TV and multimedia**

<br/>
$\textcolor{Orange}{\textbf{Efficiency Assumption}}$

improves

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

stable/decreases since 2015

efficiency counterbalances growth

In [21]:
trg_RS_ees_tvm = 0.8*ref_RS_ees_tvm # [kWh/household]
yea_RS_ees_tvm = 2050

**ICT equipment**

<br/>
$\textcolor{Orange}{\textbf{Efficiency Assumption}}$

improves

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

still continue to deploy (+9.037 kWh/household/year).

efficiency counterbalances growth

In [22]:
trg_RS_ees_ict = ref_RS_ees_ict # [kWh/household]
yea_RS_ees_ict = 2019

**Lighting**

In [23]:
trg_RS_ees_lgt = 170 # [kWh/household]
yea_RS_ees_lgt = 2030

2019+(ref_RS_ees_lgt-trg_RS_ees_lgt)/(12.853)

2031.3531395397029


> **Comment:** Degrowth from 2000 to 2023 with about -12.9 kWh/household/year. Use this to assess year target is reached.

> **Comment:** For target, see this EU labels (https://energy-efficient-products.ec.europa.eu/product-list/light-sources_en)


**Other appliances**

In [24]:
trg_RS_ees_oth = ref_RS_ees_oth # [kWh/household]
yea_RS_ees_oth = 2019


---

<font size="3"><b>2.2.2. Data Processing</b></font> <a class="anchor" id="section_2_2_2"></a>

---


In [25]:
ees_RS_tot = {
    'refrigeration':      linear_with_middle_point(2019,           ref_RS_ees_frg*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_frg, trg_RS_ees_frg*df_SUF["households [household]"][yea_RS_ees_frg]*1e-9, 
                                                   2050,           trg_RS_ees_frg*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
    'washing machines':   linear_with_middle_point(2019,           ref_RS_ees_wsh*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_wsh, trg_RS_ees_wsh*df_SUF["households [household]"][yea_RS_ees_wsh]*1e-9, 
                                                   2050,           trg_RS_ees_wsh*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
    'clothes dryers':     linear_with_middle_point(2019,           ref_RS_ees_dry*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_dry, trg_RS_ees_dry*df_SUF["households [household]"][yea_RS_ees_dry]*1e-9, 
                                                   2050,           trg_RS_ees_dry*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
    'dishwashers':        linear_with_middle_point(2019,           ref_RS_ees_dsh*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_dsh, trg_RS_ees_dsh*df_SUF["households [household]"][yea_RS_ees_dsh]*1e-9, 
                                                   2050,           trg_RS_ees_dsh*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
    'TV and multimedia':  linear_with_middle_point(2019,           ref_RS_ees_tvm*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_tvm, trg_RS_ees_tvm*df_SUF["households [household]"][yea_RS_ees_tvm]*1e-9, 
                                                   2050,           trg_RS_ees_tvm*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
    'ICT':                linear_with_middle_point(2019,           ref_RS_ees_ict*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_ict, trg_RS_ees_ict*df_SUF["households [household]"][yea_RS_ees_ict]*1e-9, 
                                                   2050,           trg_RS_ees_ict*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
    'light':              linear_with_middle_point(2019,           ref_RS_ees_lgt*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_lgt, trg_RS_ees_lgt*df_SUF["households [household]"][yea_RS_ees_lgt]*1e-9, 
                                                   2050,           trg_RS_ees_lgt*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
    'others':             linear_with_middle_point(2019,           ref_RS_ees_oth*df_SUF["households [household]"][2019]          *1e-9, 
                                                   yea_RS_ees_oth, trg_RS_ees_oth*df_SUF["households [household]"][yea_RS_ees_oth]*1e-9, 
                                                   2050,           trg_RS_ees_oth*df_SUF["households [household]"][2050]          *1e-9, years), # [TWh] kWh/household * household
}


total_TWh  = ees_RS_tot['refrigeration'][0]+ees_RS_tot['washing machines'][0]+ees_RS_tot['clothes dryers'][0]+ees_RS_tot['dishwashers'][0]\
            +ees_RS_tot['TV and multimedia'][0]+ees_RS_tot['ICT'][0]+ees_RS_tot['light'][0]+ees_RS_tot['others'][0]
total_ktoe = total_TWh*1e+3/ktoe_to_GWh
print(total_ktoe)

ees_RS_tot

997.6784178847806


{'refrigeration': [2.44, 1.935, 1.515, 1.094, 0.673, 0.683, 0.693],
 'washing machines': [0.745, 0.601, 0.482, 0.491, 0.501, 0.51, 0.519],
 'clothes dryers': [0.854, 0.738, 0.642, 0.545, 0.449, 0.455, 0.462],
 'dishwashers': [0.688, 0.723, 0.751, 0.78, 0.808, 0.837, 0.866],
 'TV and multimedia': [2.995, 2.956, 2.924, 2.891, 2.859, 2.826, 2.794],
 'ICT': [1.126, 1.163, 1.193, 1.223, 1.253, 1.284, 1.314],
 'light': [1.627, 1.236, 0.91, 0.928, 0.946, 0.963, 0.981],
 'others': [1.128, 1.164, 1.195, 1.225, 1.255, 1.285, 1.316]}


---

<font size="4"><b>2.3. Global Data Processing</b></font> <a class="anchor" id="section_2_1"></a>

---


In [26]:
df_tes_RS_tot = pd.DataFrame(tes_RS_tot, index=years)
df_ees_RS_tot = pd.DataFrame(ees_RS_tot, index=years)

df_RS_tot_activity = {
    'heat':     df_tes_RS_tot[["space heating","domestic hot water","cooking"]].T,
    'cold':     df_tes_RS_tot[["space cooling"]].T,
    'elec':     df_ees_RS_tot.T,
    }

df_RS_tot_carrier = {
    'heat':     df_tes_RS_tot[["space heating","domestic hot water"]].T,
    'cold':     df_tes_RS_tot[["space cooling"]].T,
    'elec':     pd.concat([df_tes_RS_tot[["cooking_el"]].T, df_ees_RS_tot.T], axis=0),
    'fuel_gas': df_tes_RS_tot[["cooking_ng"]].T,
    }




---

<font size="5"><b>3. Tertiary Sector</b></font> <a class="anchor" id="section_3"></a>

---



---

<font size="4"><b>3.1. Thermal Uses</b></font> <a class="anchor" id="section_3_1"></a>

---


Distribution of different energy services for thermal uses the reference year


In [27]:
# Inputs - Repartition of the thermal energy services (TES) in tertiary buildings for the reference year (2019)
ref_TS_tes_sht = 1862.949*ktoe_to_GWh*1e+6/ref_TS_sur_tot # [kWh/m²]
ref_TS_tes_scl =  336.272*ktoe_to_GWh*1e+6/ref_TS_sur_tot # [kWh/m²]
ref_TS_tes_shw =  299.836*ktoe_to_GWh*1e+6/population_dict[2019] # [kWh/person]
ref_TS_tes_cat =  284.204*ktoe_to_GWh*1e+6/population_dict[2019] # [kWh/person]
print(f'Energy service for space heating: {round(ref_TS_tes_sht,3)} kWh/m²')
print(f'Energy service for space cooling: {round(ref_TS_tes_scl,3)} kWh/m²')
print(f'Energy service for hot water:     {round(ref_TS_tes_shw,3)} kWh/person')
print(f'Energy service for catering:      {round(ref_TS_tes_cat,3)} kWh/person')

Energy service for space heating: 95.415 kWh/m²
Energy service for space cooling: 17.223 kWh/m²
Energy service for hot water:     305.045 kWh/person
Energy service for catering:      289.141 kWh/person



---

<font size="3"><b>3.1.1. Building Energy Performance</b></font> <a class="anchor" id="section_3_1_1"></a>

---

Split old and new/ren?

For old: keep it more or less constant

For new: improve?

De 2001 à 2023, le taux de modernisation du parc de bâtiments (rénovation + nouvelle construction) est de l'ordre de  1,7%/an (m²_ren/m²_tot).

<br/>
$\textcolor{Orange}{\textbf{Efficiency Assumption}}$

Renovation rate from na% to na%


In [28]:
acc_TS_tes_sht_ren = 5
cur_TS_tes_sht_ren = -0.154 # kWh/m²/year
trg_TS_tes_sht     = ref_TS_tes_sht + acc_TS_tes_sht_ren*cur_TS_tes_sht_ren*(2050-2019) # [kWh/m²]
trg_TS_tes_sht

71.54461371749028


---

<font size="3"><b>3.1.2. Behavioural Aspects</b></font> <a class="anchor" id="section_3_1_2"></a>

---

**Space heating** 

adoption of SlowHeating, report some to electric appliances!

In [29]:
d_cons_temp = 0.07
d_temp = 2
suf_TS_tes_sht = 1-d_temp*d_cons_temp


> **Comment:** From 2001 to 2023, -0.154 kWh/m²/year on average. Yet, from 2021 to 2023, following energy crisis, went from 102.6 kWh/m² to 78.6 kWh/m² (-19.3 kWh/m²/year). Shows that there us HUGE margin! And that reacihing less than 70 kWh/m² is moe than realistic!


**Space cooling** 

limit development, historical pace is quite high (+0.743 kWh/m²/year), want to reduce it. but with climate change, we have to expect growth (legal obligation!)

In [30]:
acc_TS_tes_scl_ren = 1/3
cur_TS_tes_scl_ren = 0.743 # kWh/m²/year
trg_TS_tes_scl     = ref_TS_tes_scl + acc_TS_tes_scl_ren*cur_TS_tes_scl_ren*(2050-2019) # [kWh/m²]


> **Comment:** Since COVID19 + 2022 energy crisis, stagnation (2019 was 17.3 kWh/m², while 2023 was 16.4 kWh/m²). Containing this is thus realistic!!!



**Sanitary hot water**

<br/>
$\textcolor{Orange}{\textbf{Sufficiency Assumption}}$

stop increasing!!!

In [31]:
# Inputs - Define Sufficiency Scenario Data (SUF)
pro_TS_tes_shw     = 0.00
# Outputs - Sufficiency Scenario Data (SUF)
trg_TS_tes_shw     = (1+pro_TS_tes_shw)*ref_TS_tes_shw # kWh/person
print(f'Evolution of sanitary hot water consumption: {round(pro_TS_tes_shw*100,1)}% [kWh/person]')
print(f' -> By 2019, the equivalent consumption is {round(ref_TS_tes_shw/365,2)} kWh per day and per person')
print(f' -> By 2050, the equivalent consumption is {round(trg_TS_tes_shw/365,2)} kWh per day and per person')

Evolution of sanitary hot water consumption: 0.0% [kWh/person]
 -> By 2019, the equivalent consumption is 0.84 kWh per day and per person
 -> By 2050, the equivalent consumption is 0.84 kWh per day and per person


In [32]:
# Inputs - Define Sufficiency Scenario Data (SUF)
pro_TS_tes_cat     = 0.30
ref_TS_tes_cat_gas = (8.421+106.309)/284.204 # ktoe_gas/ktoe_tot (from JRC-IDEES)
trg_TS_tes_cat_gas = 0.050
ref_TS_tes_cat_bio =          1.603 /284.204 # ktoe_bio/ktoe_tot (from JRC-IDEES)
trg_TS_tes_cat_bio = ref_TS_tes_cat_bio
# Outputs - Sufficiency Scenario Data (SUF)
trg_TS_tes_cat     = (1+pro_TS_tes_cat)*ref_TS_tes_cat # kWh/person
#trg_TS_tes_cat_gas = (1+pro_TS_tes_cat)*ref_TS_tes_cat_gas 
#trg_TS_tes_cat_bio = (1+pro_TS_tes_cat)*ref_TS_tes_cat_bio 
print(f'Evolution of catering consumption: {round(pro_TS_tes_cat*100,1)}% [kWh/person]')
print(f' -> By 2019, the equivalent consumption is {round(ref_TS_tes_cat/365,2)} kWh per day and per person')
print(f' -> By 2050, the equivalent consumption is {round(trg_TS_tes_cat/365,2)} kWh per day and per person')

Evolution of catering consumption: 30.0% [kWh/person]
 -> By 2019, the equivalent consumption is 0.79 kWh per day and per person
 -> By 2050, the equivalent consumption is 1.03 kWh per day and per person



---

<font size="3"><b>3.1.3. Centralisation Degree</b></font> <a class="anchor" id="section_3_1_3"></a>

---



---

<font size="3"><b>3.1.4. Data Processing</b></font> <a class="anchor" id="section_3_1_4"></a>

---



In [33]:
tes_TS_tot = {
    'space heating':      linear_growth(2019,ref_TS_tes_sht               *df_SUF["TS total surface [Mm²]"][2019]*1e-3,
                                        2050,trg_TS_tes_sht*suf_TS_tes_sht*df_SUF["TS total surface [Mm²]"][2050]*1e-3,years), # [TWh] kWh/m² * m²
    'space cooling':      linear_growth(2019,ref_TS_tes_scl               *df_SUF["TS total surface [Mm²]"][2019]*1e-3,
                                        2050,trg_TS_tes_scl               *df_SUF["TS total surface [Mm²]"][2050]*1e-3,years),
    'sanitary hot water': linear_growth(2019,ref_TS_tes_shw               *df_SUF["population [person]"][2019]*1e-9,
                                        2050,trg_TS_tes_shw               *df_SUF["population [person]"][2050]*1e-9,years), # [TWh] kWh/person * person 
    'catering':           linear_growth(2019,ref_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9,
                                        2050,trg_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9,years), # [TWh] kWh/person * person
    'catering_ng':        linear_growth(2019,ref_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9*ref_TS_tes_cat_gas,
                                        2050,trg_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9*trg_TS_tes_cat_gas,years), # [TWh] kWh/person * person
    'catering_bm':        linear_growth(2019,ref_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9*ref_TS_tes_cat_bio,
                                        2050,trg_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9*trg_TS_tes_cat_bio,years), # [TWh] kWh/person * person
    'catering_el':        linear_growth(2019,ref_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9*(1-ref_TS_tes_cat_gas-ref_TS_tes_cat_bio),
                                        2050,trg_TS_tes_cat               *df_SUF["population [person]"][2019]*1e-9*(1-trg_TS_tes_cat_gas-ref_TS_tes_cat_bio),years), # [TWh] kWh/person * person
}

total_TWh  = tes_TS_tot['space heating'][0]+tes_TS_tot['space cooling'][0]+tes_TS_tot['sanitary hot water'][0]+tes_TS_tot['catering'][0]
total_ktoe = total_TWh*1e+3/ktoe_to_GWh
print(total_ktoe)

# for later, centralsied vs decentralised, carriers!
tes_TS_carrier = {
    'heat': [x+y for x, y in zip(*[tes_TS_tot['space heating'],tes_TS_tot['sanitary hot water']])], # [TWh]
    'cold':                        tes_TS_tot['space cooling'],                                     # [TWh]
    'elec':                        tes_TS_tot['catering_el'],                                       # [TWh]
    'fuel_ng':                     tes_TS_tot['catering_ng'],                                       # [TWh]
	'fuel_bm':                     tes_TS_tot['catering_bm'],                                       # [TWh]
}
tes_TS_tot

2783.2330180567496


{'space heating': [21.666, 20.155, 18.897, 17.638, 16.379, 15.12, 13.861],
 'space cooling': [3.911, 4.24, 4.514, 4.788, 5.062, 5.336, 5.61],
 'sanitary hot water': [3.487, 3.556, 3.614, 3.671, 3.729, 3.786, 3.844],
 'catering': [3.305, 3.497, 3.657, 3.817, 3.977, 4.137, 4.297],
 'catering_ng': [1.334, 1.118, 0.937, 0.757, 0.576, 0.395, 0.215],
 'catering_bm': [0.019, 0.02, 0.021, 0.022, 0.022, 0.023, 0.024],
 'catering_el': [1.952, 2.36, 2.699, 3.039, 3.379, 3.718, 4.058]}


---

<font size="4"><b>3.2. Specific Electrical Uses</b></font> <a class="anchor" id="section_3_2"></a>

---


Distribution of different energy services for electrical uses the reference year


In [34]:
# Inputs - Repartition of the electrical energy services (EES) in tertiary buildings for the reference year (2019)
ref_TS_ees_vnt =  71.632*ktoe_to_GWh*1e+6/df_SUF["population [person]"][2019] # [kWh/person]
ref_TS_ees_slt =  77.819*ktoe_to_GWh*1e+6/df_SUF["population [person]"][2019] # [kWh/person]
ref_TS_ees_blt = 297.768*ktoe_to_GWh*1e+6/df_SUF["population [person]"][2019] # [kWh/person]
ref_TS_ees_frg = 179.674*ktoe_to_GWh*1e+6/df_SUF["population [person]"][2019] # [kWh/person]
ref_TS_ees_msc = 176.613*ktoe_to_GWh*1e+6/df_SUF["population [person]"][2019] # [kWh/person]
ref_TS_ees_ict = 248.983*ktoe_to_GWh*1e+6/df_SUF["population [person]"][2019] # [kWh/person]
print(f'Energy service for ventilation:      {round(ref_TS_ees_vnt,3)} kWh/person')
print(f'Energy service for street lighting:  {round(ref_TS_ees_slt,3)} kWh/person')
print(f'Energy service for building lighting:{round(ref_TS_ees_blt,3)} kWh/person')
print(f'Energy service for refrigeration:    {round(ref_TS_ees_frg,3)} kWh/person')
print(f'Energy service for miscellaneous:    {round(ref_TS_ees_msc,3)} kWh/person')
print(f'Energy service for ICT & multimedia: {round(ref_TS_ees_ict,3)} kWh/person')

Energy service for ventilation:      72.876 kWh/person
Energy service for street lighting:  79.171 kWh/person
Energy service for building lighting:302.941 kWh/person
Energy service for refrigeration:    182.795 kWh/person
Energy service for miscellaneous:    179.681 kWh/person
Energy service for ICT & multimedia: 253.308 kWh/person



**Ventilation and others**

rather stable for past 5 years

In [35]:
trg_TS_ees_vnt = ref_TS_ees_vnt # [kWh/person]

**Street lighting**

-0.476 kWh/person/year

In [36]:
trg_TS_ees_slt = ref_TS_ees_slt-0.476*(2050-2019) # [kWh/person]

**Building lighting**

In [37]:
trg_TS_ees_blt = 150 # [kWh/person]

**Commercial refrigeration**

stable

In [38]:
trg_TS_ees_frg = ref_TS_ees_frg # [kWh/person]

**Miscellaneous building technologies**

In [39]:
trg_TS_ees_msc = ref_TS_ees_msc # [kWh/person]

**ICT and multimedia** 

In [40]:
trg_TS_ees_ict = 1.10*ref_TS_ees_ict # [kWh/person]


---

<font size="3"><b>3.2.2. Data Processing</b></font> <a class="anchor" id="section_3_2_2"></a>

---


In [41]:

ees_TS_tot = {
    'ventilation':        linear_growth(2019,           ref_TS_ees_vnt*df_SUF["population [person]"][2019]          *1e-9, 
                                        2050,           trg_TS_ees_vnt*df_SUF["population [person]"][2050]          *1e-9, years), # [TWh] kWh/person * person
    'street lighting':    linear_growth(2019,           ref_TS_ees_slt*df_SUF["population [person]"][2019]          *1e-9, 
                                        2050,           trg_TS_ees_slt*df_SUF["population [person]"][2050]          *1e-9, years), # [TWh] kWh/person * person
    'building lighting':  linear_growth(2019,           ref_TS_ees_blt*df_SUF["population [person]"][2019]          *1e-9, 
                                        2050,           trg_TS_ees_blt*df_SUF["population [person]"][2050]          *1e-9, years), # [TWh] kWh/person * person
    'refrigeration':      linear_growth(2019,           ref_TS_ees_frg*df_SUF["population [person]"][2019]          *1e-9, 
                                        2050,           trg_TS_ees_frg*df_SUF["population [person]"][2050]          *1e-9, years), # [TWh] kWh/person * person
    'miscellaneous':      linear_growth(2019,           ref_TS_ees_msc*df_SUF["population [person]"][2019]          *1e-9, 
                                        2050,           trg_TS_ees_msc*df_SUF["population [person]"][2050]          *1e-9, years), # [TWh] kWh/person * person
    'ICT':                linear_growth(2019,           ref_TS_ees_ict*df_SUF["population [person]"][2019]          *1e-9, 
                                        2050,           trg_TS_ees_ict*df_SUF["population [person]"][2050]          *1e-9, years), # [TWh] kWh/person * person
}


total_TWh  = ees_TS_tot['ventilation'][0]+ees_TS_tot['street lighting'][0]+ees_TS_tot['building lighting'][0]+ees_TS_tot['refrigeration'][0]\
            +ees_TS_tot['miscellaneous'][0]+ees_TS_tot['ICT'][0]
total_ktoe = total_TWh*1e+3/ktoe_to_GWh
print(total_ktoe)

ees_TS_tot

1052.536543422184


{'ventilation': [0.833, 0.85, 0.863, 0.877, 0.891, 0.905, 0.918],
 'street lighting': [0.905, 0.887, 0.872, 0.857, 0.842, 0.827, 0.812],
 'building lighting': [3.463, 3.159, 2.905, 2.651, 2.398, 2.144, 1.89],
 'refrigeration': [2.09, 2.131, 2.165, 2.2, 2.234, 2.269, 2.303],
 'miscellaneous': [2.054, 2.095, 2.129, 2.162, 2.196, 2.23, 2.264],
 'ICT': [2.896, 3.015, 3.114, 3.213, 3.313, 3.412, 3.511]}


---

<font size="4"><b>3.3. Global Data Processing</b></font> <a class="anchor" id="section_3_3"></a>

---


In [52]:
df_tes_TS_tot = pd.DataFrame(tes_TS_tot, index=years)
df_ees_TS_tot = pd.DataFrame(ees_TS_tot, index=years)

df_TS_tot_activity = {
    'heat':     df_tes_TS_tot[["space heating","sanitary hot water","catering"]].T,
    'cold':     df_tes_TS_tot[["space cooling"]].T,
    'elec':     df_ees_TS_tot.T,
    }

df_TS_tot_carrier = {
    'heat':     df_tes_TS_tot[["space heating","sanitary hot water"]].T,
    'cold':     df_tes_TS_tot[["space cooling"]].T,
    'elec':     pd.concat([df_tes_TS_tot[["catering_el"]].T, df_ees_TS_tot.T], axis=0),
    'fuel_gas': df_tes_TS_tot[["catering_ng"]].T,
    'fuel_bio': df_tes_TS_tot[["catering_bm"]].T,
    }

df_TS_tot_carrier['elec'].sum()

2019    14.193
2025    14.497
2030    14.747
2035    14.999
2040    15.253
2045    15.505
2050    15.756
dtype: float64


---

<font size="5"><b>4. Global Data Processing</b></font> <a class="anchor" id="section_4"></a>

---




In [65]:
df_TS_tot_carrier 
df_TS_tot_carrier

BS_tot_carrier = {
    'heat':      df_RS_tot_carrier['heat'].sum()\
                +df_TS_tot_carrier['heat'].sum(), # [TWh]
    'cold':      df_RS_tot_carrier['cold'].sum()\
                +df_TS_tot_carrier['cold'].sum(), # [TWh]
    'elec':      df_RS_tot_carrier['elec'].sum()\
                +df_TS_tot_carrier['elec'].sum(), # [TWh]
    'fuel_gas':  df_RS_tot_carrier['fuel_gas'].sum()\
                +df_TS_tot_carrier['fuel_gas'].sum(), # [TWh]
	'fuel_bio':  df_TS_tot_carrier['fuel_bio'].sum(), # [TWh]
}

df_BS_tot_carrier = pd.DataFrame(BS_tot_carrier, index=years)

df_BS_tot_carrier.T


,2019,2025,2030,2035,2040,2045,2050
heat,75.482,69.701,64.885,60.068,55.251,50.433,45.616
cold,4.184,4.643,5.025,5.407,5.789,6.171,6.554
elec,26.617,25.960,25.412,25.334,25.260,25.716,26.174
fuel_gas,1.634,1.366,1.141,0.918,0.693,0.469,0.245
fuel_bio,0.019,0.020,0.021,0.022,0.022,0.023,0.024


In [67]:
eud_RS_tes = 4447.334 # ktoe
eud_RS_ees =  997.764 # ktoe
eud_TS_tes = 2783.260 # ktoe
eud_TS_ees = 1052.489 # ktoe
eud        = eud_RS_tes + eud_RS_ees + eud_TS_tes + eud_TS_ees
print(df_BS_tot_carrier.T.sum()[2019])

107.936
